In [10]:
# import csv

# csv_file_name = 'MS_data.csv'

# columns = ["Title", "Authors", "Publish Date", "Abstract", "Keywords", "PDF Name"]
# with open(csv_file_name, mode='w', newline='', encoding='utf-8') as file:
#     writer = csv.DictWriter(file, fieldnames=columns)
#     writer.writeheader()

# def add_row_to_csv(data):
#     with open(csv_file_name, mode='a', newline='', encoding='utf-8') as file:
#         writer = csv.DictWriter(file, fieldnames=columns)
#         writer.writerow(data)

In [11]:
import csv

csv_file_name = 'MS_data.csv'

columns = ["Title", "Authors", "Publish Date", "Abstract", "Keywords", "PDF Name"]

def add_row_to_csv(data):
    with open(csv_file_name, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=columns)
        writer.writerow(data)

In [12]:
def get_credentials():
    with open('../credentials.txt', 'r') as file:
        lines = file.readlines()
        username = lines[0].strip()
        password = lines[1].strip()
        file.close()
        return username, password


In [13]:
def overwrite_line(line_number, new_value):
    with open('progress.txt', 'r') as file:
        lines = file.readlines()
        
    if 0 <= line_number < len(lines):
        lines[line_number] = f"{new_value}\n" 
        with open('progress.txt', 'w') as file:
            file.writelines(lines)
        return True
    else:
        return False

def read_numbers():
    with open('progress.txt', 'r') as file:
        numbers = [int(line.strip()) for line in file]
    return numbers[0], numbers[1], numbers[2], numbers[3]

In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
import requests
import time
import urllib
import os
import shutil
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Initial_path = r"C:\Users\yitong\Desktop\Research-Assistant\Scrape\MS\MS_PDF"

In [15]:
def get_one_article(driver, wait):

    # Function to find the following dd for a given dt text
    def find_following_dd_by_dt_text(dt_text):
        dt_elements = driver.find_elements(By.XPATH, "//dl[@id='citationFields']/dt")
        for dt in dt_elements:
            if dt.text.strip(':') == dt_text:
                return dt.find_element(By.XPATH, "./following-sibling::dd[1]")
        return None

    title = None
    authors = None
    subject_terms = None
    publish_date = None
    accession_number = None
    abstract = None

    # Extract Title
    try:
        title_element = driver.find_element(By.CSS_SELECTOR, "dd.citation-title > a > div > h1")
        title = title_element.text
    except:
        pass

    # Extract Authors
    try:
        authors_element = find_following_dd_by_dt_text("Authors")
        authors = "; ".join([author.text for author in authors_element.find_elements(By.TAG_NAME, "a")])
    except:
        pass

    # Extract Subject Terms
    try:
        subject_terms_element = find_following_dd_by_dt_text("Subject Terms")
        subject_terms = "; ".join([term.text for term in subject_terms_element.find_elements(By.TAG_NAME, "a")])
    except:
        pass

    # Extract Publish Date (assuming it's within the Source information)
    try:
        source_element = find_following_dd_by_dt_text("Source")
        publish_date = source_element.text.split('. ')[1].split(', ')[0]
    except:
        pass

    # Extract Accession Number
    try:
        accession_number_element = find_following_dd_by_dt_text("Accession Number")
        accession_number = accession_number_element.text
    except:
        pass

    try:
        abstract_element = find_following_dd_by_dt_text("Abstract")
        abstract = abstract_element.text
    except:
        pass

    add_row_to_csv({
        "Title": title,
        "Authors": authors,
        "Publish Date": publish_date,
        "Abstract": abstract,
        "Keywords": subject_terms,
        "PDF Name": accession_number
    })

    try:
        pdf_link_element = driver.find_element("link text", 'PDF Full Text')
        pdf_link_element.click()
        wait.until(EC.element_to_be_clickable((By.ID, "downloadLink")))
        # time.sleep(5)
    
        pdf_download_element = driver.find_element(By.ID, "downloadLink")
        pdf_download_element.click()
        time.sleep(20)
        try:
            filename = max([Initial_path + "\\" + f for f in os.listdir(Initial_path)], key=os.path.getctime)
            shutil.move(filename, os.path.join(Initial_path, accession_number + ".pdf"))
        # driver.execute_script("window.history.go(-1)")
        # time.sleep(5)
        except PermissionError:
            time.sleep(60)
            filename = max([Initial_path + "\\" + f for f in os.listdir(Initial_path)], key=os.path.getctime)
            shutil.move(filename, os.path.join(Initial_path, accession_number + ".pdf"))
        result_list_element = driver.find_element(By.CSS_SELECTOR, ".nav-item .resultListLink")
        result_list_element.click()
    except NoSuchElementException:
        try:
            NUS_download = driver.find_element("xpath", "//img[contains(@alt, 'Find It! @ NUS Libraries')]")
            NUS_download.click()
            window_handles = driver.window_handles
            driver.switch_to.window(window_handles[1])
            wait.until(EC.presence_of_element_located(("link text", 'Click to get Full Text')))
            link_element = driver.find_element("link text", 'Click to get Full Text')
            link_element.click()
            window_handles = driver.window_handles
            driver.switch_to.window(window_handles[2])
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h5.hlFld-Title.meta__title.meta__title__margin > a")))
            download_element = driver.find_element(By.CSS_SELECTOR, "h5.hlFld-Title.meta__title.meta__title__margin > a")
            download_element.click()
            wait.until(EC.presence_of_element_located(("link text", 'View PDF')))
            pdf_element = driver.find_element("link text", 'View PDF')
            pdf_element.click()
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[data-download-files-key='pdf']")))
            pdf_links = driver.find_elements(By.CSS_SELECTOR, "a[data-download-files-key='pdf']")
            for link in pdf_links:
                url = link.get_attribute('href')
                driver.get(url)
                break
            time.sleep(5)

            driver.close()
            driver.switch_to.window(window_handles[1])
            driver.close()
            driver.switch_to.window(window_handles[0])
            time.sleep(20)
            try:
                filename = max([Initial_path + "\\" + f for f in os.listdir(Initial_path)], key=os.path.getctime)
                shutil.move(filename, os.path.join(Initial_path, accession_number + ".pdf"))
            except PermissionError:
                time.sleep(60)
                filename = max([Initial_path + "\\" + f for f in os.listdir(Initial_path)], key=os.path.getctime)
                shutil.move(filename, os.path.join(Initial_path, accession_number + ".pdf"))
            # driver.execute_script("window.history.go(-1)")
            # time.sleep(5)
            result_list_element = driver.find_element(By.ID, "ctl00_ctl00_MainContentArea_MainContentArea_topNavControl_linkResult")
            result_list_element.click()
        except:
            window_handles = driver.window_handles
            driver.switch_to.window(window_handles[2])
            driver.close()
            driver.switch_to.window(window_handles[1])
            driver.close()

    except:
        # go back to result list
        # print("No NUS download")
        time.sleep(5)
        element = driver.find_element(By.ID, "ctl00_ctl00_MainContentArea_MainContentArea_topNavControl_linkResult")
        element.click()
    
    time.sleep(5)


In [16]:
# 先处理这个年份这个issue剩下的article
# 再处理这个年份剩下的issue
# 再处理剩下的年份

def recover_from_breakpoint(year, issue, page, article):
    
    user_id, password = get_credentials()

    options = webdriver.ChromeOptions()
    options.add_experimental_option('prefs', {
        "download.default_directory": Initial_path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": True 
    })

    wd = webdriver.Chrome(service = Service(r'../chromedriver.exe'), options=options)
    wait = WebDriverWait(wd, 60)
    
    wd.get("https://libproxy1.nus.edu.sg/login?url=https://search.ebscohost.com/login.aspx?direct=true&db=bsu&jid=MGS&site=ehost-live")
    button = wd.find_element(By.ID, 'btn_nus')
    button.click()
    wait.until(EC.element_to_be_clickable((By.ID, 'userNameInput')))
    user_id_box = wd.find_element(By.ID, 'userNameInput')
    password_box = wd.find_element(By.ID, 'passwordInput')
    user_id_box.send_keys(user_id)
    password_box.send_keys(password)
    submit_button = wd.find_element(By.ID, 'submitButton')
    submit_button.click()
    wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "osano-cm-accept-all")))
    cookie_button_element = wd.find_element(By.CLASS_NAME, "osano-cm-accept-all")
    cookie_button_element.click()

    year_xpath = "//td[@class='authVolIssue_volume_cell']/a[contains(@href, 'javascript:ToggleVolumeIssue')]"
    wait.until(EC.element_to_be_clickable((By.XPATH, year_xpath)))
    year_elements = wd.find_elements(By.XPATH, year_xpath)
    year_elements_len = len(year_elements)
    for j in range(year, year_elements_len):
        overwrite_line(0, j)
        year_elements[j].click()
        time.sleep(3)
        issue_xpath = "//a[@data-auto='volume_issue_issue_link' and contains(@href, 'XslPostBack')]"
        wait.until(EC.element_to_be_clickable((By.XPATH, issue_xpath)))
        issue_elements = wd.find_elements(By.XPATH, issue_xpath)
        issue_elements_len = len(issue_elements)
        for i in range(issue, issue_elements_len):
            overwrite_line(1, i)
            issue_elements[i].click()
            time.sleep(5)
            if page == 1:
                title_elements = wd.find_elements(By.CSS_SELECTOR, "a.title-link.color-p4")
                for k in range(article, len(title_elements)):
                    overwrite_line(3, k)
                    title_elements[k].click()
                    print(j, i, page, k)
                    time.sleep(5)
                    get_one_article(wd, wait)
                    window_handles = wd.window_handles
                    wd.switch_to.window(window_handles[0])
                    title_elements = wd.find_elements(By.CSS_SELECTOR, "a.title-link.color-p4")
                    # overwrite_line(3, k + 1)

                page_element = wd.find_elements(By.XPATH, "//a[@title='Skip to page 2']")
                if len(page_element) == 0:
                    # This issue only has 1 page
                    page = 1
                    article = 0
                    overwrite_line(2, page)
                    overwrite_line(3, 0)
                else:
                    page = 2
                    article = 0
                    overwrite_line(2, page)
                    overwrite_line(3, 0)

            if page == 2:
                page_element = wd.find_elements(By.XPATH, "//a[@title='Skip to page 2']")
                page_element[0].click()
                title_elements = wd.find_elements(By.CSS_SELECTOR, "a.title-link.color-p4")
                for m in range(article, len(title_elements)):
                    overwrite_line(3, m)
                    title_elements[m].click()
                    print(j, i, page, m)
                    time.sleep(5)
                    get_one_article(wd, wait)
                    window_handles = wd.window_handles
                    wd.switch_to.window(window_handles[0])
                    title_elements = wd.find_elements(By.CSS_SELECTOR, "a.title-link.color-p4")
                    # overwrite_line(3, m + 1)

                page_element = wd.find_elements(By.XPATH, "//a[@title='Skip to page 3']")
                if len(page_element) == 0:
                    # This issue only has 2 pages
                    page = 1
                    article = 0
                    overwrite_line(2, page)
                    overwrite_line(3, 0)
                else:
                    page = 3
                    article = 0
                    overwrite_line(2, page)
                    overwrite_line(3, 0)

            if page == 3:
                page_element = wd.find_elements(By.XPATH, "//a[@title='Skip to page 3']")
                page_element[0].click()
                title_elements = wd.find_elements(By.CSS_SELECTOR, "a.title-link.color-p4")
                for n in range(article, len(title_elements)):
                    overwrite_line(3, n)
                    title_elements[n].click()
                    print(j, i, page, n)
                    time.sleep(5)
                    get_one_article(wd, wait)
                    window_handles = wd.window_handles
                    wd.switch_to.window(window_handles[0])
                    title_elements = wd.find_elements(By.CSS_SELECTOR, "a.title-link.color-p4")
                page_element = wd.find_elements(By.XPATH, "//a[@title='Skip to page 4']")
                if len(page_element) == 0:
                    # This issue only has 3 page
                    page = 1
                    article = 0
                    overwrite_line(2, page)
                    overwrite_line(3, 0)
                else:
                    page = 4
                    article = 0
                    overwrite_line(2, page)
                    overwrite_line(3, 0)
            if page == 4:
                page_element = wd.find_elements(By.XPATH, "//a[@title='Skip to page 4']")
                page_element[0].click()
                title_elements = wd.find_elements(By.CSS_SELECTOR, "a.title-link.color-p4")
                for n in range(article, len(title_elements)):
                    overwrite_line(3, n)
                    title_elements[n].click()
                    print(j, i, page, n)
                    time.sleep(5)
                    get_one_article(wd, wait)
                    window_handles = wd.window_handles
                    wd.switch_to.window(window_handles[0])
                    title_elements = wd.find_elements(By.CSS_SELECTOR, "a.title-link.color-p4")
                    
            page = 1
            article = 0
            overwrite_line(2, page)
            overwrite_line(3, 0)
            # overwrite_line(1, i + 1)
            wd.get("https://libproxy1.nus.edu.sg/login?url=https://search.ebscohost.com/login.aspx?direct=true&db=bsu&jid=MGS&site=ehost-live")
            time.sleep(10)
            try:
                button = wd.find_element(By.ID, 'btn_nus')
                button.click()
            except NoSuchElementException:
                pass
            time.sleep(10)
            year_elements = wd.find_elements(By.XPATH, year_xpath)
            year_elements[j].click()
            time.sleep(5)
            issue_elements = wd.find_elements(By.XPATH, issue_xpath)
            print("finish issue")
        issue = 0
        overwrite_line(1, 0)
        print("finish year")
        # overwrite_line(0, j + 1)


In [17]:
import time
temp = time.time()
while True:
    try:
        year, issue, page, article = read_numbers()
        recover_from_breakpoint(year, issue, page, article)
    except Exception as e:
        print(e)

20 9 1 0
20 9 1 1
20 9 1 2
20 9 1 3
20 9 1 4
20 9 1 5
20 9 1 6
20 9 1 7
20 9 1 8
20 9 1 9
20 9 2 0
20 9 2 1
20 9 2 2
finish issue
20 10 1 0
20 10 1 1
20 10 1 2
20 10 1 3
20 10 1 4
20 10 1 5
20 10 1 6
20 10 1 7
20 10 1 8
20 10 1 9
20 10 2 0
finish issue
20 11 1 0
20 11 1 1
20 11 1 2
20 11 1 3
20 11 1 4
20 11 1 5
20 11 1 6
20 11 1 7
20 11 1 8
20 11 1 9
finish issue
20 12 1 0
20 12 1 1
20 12 1 2
20 12 1 3
20 12 1 4
20 12 1 5
20 12 1 6
20 12 1 7
20 12 1 8
finish issue
finish year
21 0 1 0
21 0 1 1
21 0 1 2
21 0 1 3
21 0 1 4
21 0 1 5
21 0 1 6
21 0 1 7
21 0 1 8
21 0 1 9
finish issue
21 1 1 0
21 1 1 1
21 1 1 2
21 1 1 3
21 1 1 4
21 1 1 5
21 1 1 6
21 1 1 7
21 1 1 8
21 1 1 9
finish issue
21 2 1 0
21 2 1 1
21 2 1 2
21 2 1 3
21 2 1 4
21 2 1 5
21 2 1 6
21 2 1 7
21 2 1 8
finish issue
21 3 1 0
21 3 1 1
21 3 1 2
21 3 1 3
21 3 1 4
21 3 1 5
21 3 1 6
21 3 1 7
21 3 1 8
21 3 1 9
21 3 2 0
finish issue
21 4 1 0
21 4 1 1
21 4 1 2
21 4 1 3
21 4 1 4
21 4 1 5
21 4 1 6
21 4 1 7
21 4 1 8
finish issue
21 5 1 0
21 5